In [6]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from gensim.models import Word2Vec

In [7]:
# Load the dataset
df = pd.read_csv("../../data/cleaned/out.csv")

# Text preprocessing
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    return text

df['text'] = df['text'].apply(preprocess_text)


In [8]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(remove_stopwords)


[nltk_data] Downloading package stopwords to /home/juan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
tokenized_texts = [text.split() for text in df['text']]
word2vec = Word2Vec(tokenized_texts, vector_size=50, window=5, min_count=1)
print(word2vec.wv['good'])

[-1.0002824   0.50857365 -1.5845125  -1.0070028   1.0680755  -0.02567383
  0.24009354  3.3515723  -1.6856357  -0.94262326  0.36953592 -2.4019828
  0.87575024  1.4228108  -0.35064992  0.7064087   1.7947127  -0.78772587
 -1.4907619  -0.7764842   1.3168302   1.5367247   2.0797865   0.8773612
 -0.80978525  1.8394427  -1.0109826   0.08838938 -0.11758807  0.27711758
 -0.19220321  0.03485214 -1.1287682  -0.35237545  0.0869664   2.6283176
  0.3283814  -1.8839879   0.25753117 -0.09678795  1.8077937  -0.32725492
  0.5822203   0.15976019  1.3051248  -0.18149598 -1.3326164  -1.8458551
  1.5937154   0.07723075]


In [14]:
def text_to_vector(text):
    words = text.split()
    word_vectors = [word2vec.wv[word] for word in words]
    return np.mean(word_vectors, axis=0)

df['vector'] = df['text'].apply(text_to_vector)

/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [15]:
encoder = OneHotEncoder(sparse=False)
encoded_labels = encoder.fit_transform(df['label'].values.reshape(-1, 1))


/home/juan/anaconda3/envs/SentAnalysis/lib/python3.8/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])